In [18]:
import pandas as pd

matchup_code = 'R6CH' # championship matchup
year = 2024

bracket_sims = pd.read_csv('./bracket_simulations.csv')

m_bracket_sims = bracket_sims[bracket_sims['Tournament'] == 'M']
# w_bracket_sims = bracket_sims[bracket_sims['Tournament'] == 'W']
    
seeds_m = pd.read_csv("march-machine-learning-mania-2025/MNCAATourneySeeds.csv")

# Have to fix X16, Y10, Y16, Z10 (change for 2025 seeds when we get them)
# X16 = Wagner = 1447
seeds_m = pd.concat([seeds_m, pd.DataFrame([{'Season': 2024, 'Seed': 'X16', 'TeamID': 1447}])], ignore_index=True)
# Y10 = Colorado St = 1161
seeds_m = pd.concat([seeds_m, pd.DataFrame([{'Season': 2024, 'Seed': 'Y10', 'TeamID': 1161}])], ignore_index=True)
# Y16 = Grambling = 1212
seeds_m = pd.concat([seeds_m, pd.DataFrame([{'Season': 2024, 'Seed': 'Y16', 'TeamID': 1212}])], ignore_index=True)
# Z10 = Colorado = 1160
seeds_m = pd.concat([seeds_m, pd.DataFrame([{'Season': 2024, 'Seed': 'Z10', 'TeamID': 1160}])], ignore_index=True)

teams = pd.read_csv("march-machine-learning-mania-2025/MTeams.csv")
seeds_m = seeds_m.merge(teams[['TeamID', 'TeamName']], on='TeamID')
seeds_m = seeds_m[seeds_m['Season'] == year]

seeds_m.head()

# Determining how many brackets were generated
num_brackets = bracket_sims['Bracket'].max()

# Labeling teams with the percentages of winning a particular matchup, given how many times they won in the simulation
value_counts = m_bracket_sims[m_bracket_sims['Slot'] == matchup_code]['Team'].value_counts()
percentages = (value_counts / num_brackets) * 100
team_names = seeds_m.set_index('Seed')['TeamName']
percentages.index = percentages.index.map(team_names)

# Represents the conditional probability that a team wins a particular matchup
percentages


Team
Connecticut       20.218
Houston           14.768
Purdue            11.925
Arizona            8.344
Iowa St            7.394
North Carolina     5.955
Tennessee          3.953
Creighton          2.781
Marquette          2.756
Illinois           2.639
Duke               2.596
Gonzaga            2.379
St Mary's CA       1.995
Baylor             1.953
Kentucky           1.481
Auburn             1.297
Wisconsin          0.830
South Carolina     0.662
San Diego St       0.580
Alabama            0.577
Clemson            0.554
BYU                0.507
Kansas             0.460
Texas              0.392
Texas Tech         0.333
Washington St      0.331
Michigan St        0.270
Florida            0.267
Mississippi St     0.188
NC State           0.181
Dayton             0.148
Northwestern       0.147
Nebraska           0.132
Oregon             0.132
Texas A&M          0.113
Utah St            0.106
TCU                0.099
Colorado           0.099
Colorado St        0.083
FL Atlantic        0

In [19]:
# Make a copy of bracket_simulations.csv FIRST, name it remaining_perfect_brackets.csv

def count_perfect_brackets(team_id, slot):
    """
    Count the number of perfect brackets remaining for a given team ID and slot.
    
    Parameters:
    team_id (int): The ID of the team.
    slot (str): The slot corresponding to a particular matchup.
    
    Returns:
    int: The number of perfect brackets remaining.
    """
    # Load the remaining perfect brackets
    remaining_brackets = pd.read_csv('remaining_perfect_brackets.csv')
    
    # Filter the bracket simulations for the given slot and team ID
    matching_brackets = m_bracket_sims[(m_bracket_sims['Slot'] == slot) & (m_bracket_sims['Team'] == team_id)]
    
    # Get the list of brackets that have the given team ID and slot
    matching_bracket_ids = matching_brackets['Bracket'].unique()
    
    # Remove brackets that don't have this pick from the remaining perfect brackets
    remaining_brackets = remaining_brackets[remaining_brackets['Bracket'].isin(matching_bracket_ids)]
    
    # Save the updated remaining perfect brackets
    remaining_brackets.to_csv('remaining_perfect_brackets.csv', index=False)
    
    # Return the count of remaining perfect brackets
    return remaining_brackets['Bracket'].nunique()

# Example usage:
team_id = 'W02'  # Replace with the desired team seed identifier (this is Uconn)
slot = 'R6CH'  # Replace with the desired slot (this is the championship)
perfect_brackets = 0

try:
    perfect_brackets = count_perfect_brackets(team_id, slot)
except FileNotFoundError:
    print("Make sure you have a copy of bracket_simulations.csv named remaining_perfect_brackets.csv")
    print()
    perfect_brackets = 0

print(f"Percent of perfect brackets remaining for team ID {team_id} in slot {slot}: {(perfect_brackets / num_brackets)*100}%")

Make sure you have a copy of bracket_simulations.csv named remaining_perfect_brackets.csv

Percent of perfect brackets remaining for team ID W02 in slot R6CH: 0.0%
